## No vector computing

In [1]:
import compute_std
import function_set_std

In [2]:
alias_values = {'1AJ': 30000.}

In [3]:
input_values = compute_std.prepare(alias_values)

In [4]:
compute_std.compute(input_values)

KeyError: 'inverse'

In [ ]:
%timeit compute_std.compute(input_values)

## Vector computing

In [ ]:
import compute_np
import function_set_np

In [ ]:
alias_values = {'1AJ': 30000.}

In [ ]:
input_values = compute_np.prepare(alias_values)

In [ ]:
compute_np.compute(input_values)

In [ ]:
%timeit compute_np.compute(input_values)